In [1]:
!pip install langchain bitsandbytes accelerate langchain_community sentence-transformers faiss-gpu pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 2.6 MB/s eta 0:00:00
  Using cached nvidi

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain import HuggingFaceHub
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.chains import RetrievalQA

import torch
import os
import warnings

warnings.filterwarnings("ignore")

In [88]:
pdf_file = PyPDFLoader("/content/document.pdf")
pages = pdf_file.load_and_split()

In [89]:
print(len(pages))

print(pages[1])

10
page_content='i) MYK: Mesleki Yeterlilik Kurumunu,\nj) MYK mesleki yeterlilik belgesi: MYK tarafından onaylanmış, bireyin bilgi, beceri ve yetkinliğini ifade eden mesleki yeterlilik belgesini,\nk) Onaylı tren makinist bröve sureti: Demiryolu tren işletmecisi veya altyapı işletmecisi tarafından düzenlenen, tren makinist brövesinde yer alan\nbilgilerin düz yazı biçiminde yer aldığı belgeyi,\nl) Otomotris: Üzerinde bulunan motor tarafından üretilen tahrik gücü ile hareket eden, gerektiğinde arkasına ve önüne bağlı olan, çekilen araçları\nhareket ettiren ve/veya aynı zamanda üzerinde yolcu veya yük taşınmasına olanak sağlayan raylı sistem aracını,\nm) Uyuşturucu ve uyarıcı maddeler: Merkezi sinir sisteminde etkisini gösteren ve beynin işlevlerini değiştirerek algıda, ruh hâlinde, bilinçte ve\ndavranışta geçici veya kalıcı değişikliklere neden olan her türlü maddeyi,\nn) Psikoteknik değerlendirme: Bireyin belirli bir işteki yeterliliğinin ortaya konması amacıyla, gerekli bilişsel ve moto

In [90]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1024,
    chunk_overlap = 64
)

chunks = text_splitter.split_documents(pages)

print(len(chunks))

print(chunks[1].page_content)

40
kişileri,
kapsamaz.
Dayanak
MADDE 2 –
 (Değişik:RG-18/5/2019-30778)
(1) Bu Yönetmelik; 10/7/2018 tarihli ve 30474 sayılı Resmî Gazete’de yayımlanan 1 sayılı Cumhurbaşkanlığı Teşkilatı Hakkında
Cumhurbaşkanlığı Kararnamesinin 478 inci maddesinin birinci fıkrasının (a) ve (d) bentleri ile 26/9/2011 tarihli ve 655 sayılı Ulaştırma ve Altyapı
Alanına İlişkin Bazı Düzenlemeler Hakkında Kanun Hükmünde Kararnamenin 28 inci maddesine dayanılarak hazırlanmıştır.
Tanımlar
MADDE 3 –
 
(Değişik:RG-30/12/2017-30286 Mükerrer)
(1) Bu Yönetmelikte geçen;
a)  
(Değişik:RG-18/5/2019-30778)
 Bakan: Ulaştırma ve Altyapı Bakanını,
b) 
(Değişik:RG-18/5/2019-30778)
 Bakanlık: Ulaştırma ve Altyapı Bakanlığını,
c) Belgelendirme: Personelin belirli bir standart veya teknik düzenlemeye uygun olduğunun, yazılı olarak bağımsız bir kurum veya kuruluş
tarafından belirlenmesi ve belgelendirilmesi faaliyetini,


In [91]:
Embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
                                   model_kwargs={"device": "cuda"})

vector_db = FAISS.from_texts([str(chunk.page_content) for chunk in chunks], Embeddings)

In [92]:
vector_db.embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='sentence-transformers/paraphrase-multilingual-mpnet-base-v2', cache_folder=None, model_kwargs={'device': 'cuda'}, encode_kwargs={}, multi_process=False, show_progress=False)

In [95]:
import pandas as pd

In [140]:
questions = [
    "Tren makinist ehliyeti almak için gereken asgari şartlar ve belgeler nelerdir?",
    "Otomotris nedir?",
    "Hangi durumlarda tren makinistinin ehliyeti askıya alınır?",
    "Hangi durumlar ciddi kaza olarak sınıflandırılır?",
    "TCDD'nin açılımı nedir?",
    "Makinist brövesi nedir?",
    "Tren makinist brövesinin geçerliliğinin devam edebilmesi için hangi sınavları geçmelidir?"
]

In [141]:
questions

['Tren makinist ehliyeti almak için gereken asgari şartlar ve belgeler nelerdir?',
 'Otomotris nedir?',
 'Hangi durumlarda tren makinistinin ehliyeti askıya alınır?',
 'Hangi durumlar ciddi kaza olarak sınıflandırılır?',
 "TCDD'nin açılımı nedir?",
 'Makinist brövesi nedir?',
 'Tren makinist brövesinin geçerliliğinin devam edebilmesi için hangi sınavları geçmelidir?']

In [142]:
relevant_results = vector_db.similarity_search(questions[-1], k=3)

In [143]:
relevant_results

[Document(page_content='aldıkları eğitimlerin ve başarılı oldukları sınavların belgelendirilmeleri şartıyla bir defaya mahsus olmak üzere işletme tarafından makiniste çalıştığı hat ve\nkullandığı cer aracına ait bröve düzenlenir.\nTren makinistinin psikoteknik kontrolleri'),
 Document(page_content='(4) Bir tren makinistinin eğitim ve sınavlar sonucunda altyapı ve cer araçları ile ilgili yeni yeterlilikler kazanması halinde, kazanılan yeterlilikler en\ngeç beş iş günü içerisinde demiryolu altyapı işletmecisi veya tren işletmecileri tarafından makinist brövesine işlenir.\nTren makinist brövesi biçimi\nMADDE 15 –\n \n(1) Tren makinist brövesinin biçimi Ek-6’da verilen ölçütlere göre düzenlenir.\nTren makinist brövesi geçerliliği\nMADDE 16 –\n \n(1) Tren makinist brövesinin geçerliliğinin devam edebilmesi için, \n \ntren makinisti aşağıda verilen zaman aralıklarında yapılacak\nsınavlarda başarılı olmak zorundadır:\na) \n(Değişik:RG-18/5/2019-30778)\n Cer aracı bilgisi: Her beş yılda bir ve

In [144]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", quantization_config=bnb_config)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [145]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)
lc_pipeline = HuggingFacePipeline(pipeline=pipe)

In [149]:
prompt = """
Aşaıdaki bilgiyi kullanarak:
\n
{context}
\n
Aşağıdaki soruyu farklı kelimeler kullanarak cevapla:
\
{question}
\n
Cevap:
\n
"""

prompt = ChatPromptTemplate.from_template(prompt)

In [150]:
qa_chain = RetrievalQA.from_chain_type(
    llm = lc_pipeline,
    retriever = vector_db.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs = {"prompt": prompt},
    return_source_documents=True
)

result = qa_chain({"query": questions[-1]})["result"]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [151]:
print(result.split("Cevap:")[1])




Tren makinist, her beş yılda bir veya her yeni araç kullanıma alındığında veya ilgili güzergâh üzerinde on sekiz aydan fazla verilen herhangi bir aradan sonra, cer aracı bilgisi sınavında başarılı olmalıdır. Ayrıca, her beş yılda bir veya yeni bir hat açıldığında veya ilgili güzergâh üzerinde on sekiz aydan fazla verilen herhangi bir aradan sonra, güzergâh ve işletme kuralları dâhil altyapı bilgisi sınavında başarılı olmalıdır.
